## Problem Statement

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business goal. To reduce customer churn, telecom companies need to predict which customers are at high risk of churn. In this project, you will analyze customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn.

In this competition, your goal is to build a machine learning model that is able to predict churning customers based on the features provided for their usage.

## File Descriptions

- train.csv: Contains 172 columns. The primary key that represents each customer is id. The target variable that you need to predict is churn_probability which contains a value of 0 or 1. This data is what you are going to use for EDA, cleaning, feature engineering, model building, model evaluation, model selection, and finally model training.

- test.csv: Contains 171 columns, doesn't contain the target variable churn_probability. You will apply all the necessary preprocessing steps to get this data into the right format and then use the model trained using the train.csv file to make predictions with this. This is unseen data! Only the competition hosts know the actual values of the target feature for this data and therefore your submissions will be evaluated on how well your model does with this dataset.

- sample.csv: This contains the format in which you need to submit the solutions to Kaggle. The id column in this dataset exactly the same as the id column in test.csv. You will make your predictions on the test.csv data and store them in a submission file that has the same format as this file. Check the Overview>Evaluation tab for mode details.

- data_dictionary.csv: This contains the definitions for the various acronyms that you will need to understand each variable. For example, the variable total_og_mou_7, contains the acronyms total, og, mou, and 7, which can be translated as the total outgoing minutes of voice calls made by the user in month of July.

## Data Definitions
The definitions are also listed down below:

    CIRCLE_ID : Telecom circle area to which the customer belongs to
    LOC : Local calls - within same telecom circle
    STD : STD calls - outside the calling circle
    IC : Incoming calls
    OG : Outgoing calls
    T2T : Operator T to T, i.e. within same operator (mobile to mobile)
    T2M : Operator T to other operator mobile
    T2O : Operator T to other operator fixed line
    T2F : Operator T to fixed lines of T
    T2C : Operator T to it’s own call center
    ARPU : Average revenue per user
    MOU : Minutes of usage - voice calls
    AON : Age on network - number of days the customer is using the operator T network
    ONNET : All kind of calls within the same operator network
    OFFNET : All kind of calls outside the operator T network
    ROAM : Indicates that customer is in roaming zone during the call
    SPL : Special calls
    ISD : ISD calls
    RECH : Recharge
    NUM : Number
    AMT : Amount in local currency
    MAX : Maximum
    DATA : Mobile internet
    3G : 3G network
    AV : Average
    VOL : Mobile internet usage volume (in MB)
    2G : 2G network
    PCK : Prepaid service schemes called - PACKS
    NIGHT : Scheme to use during specific night hours only
    MONTHLY : Service schemes with validity equivalent to a month
    SACHET : Service schemes with validity smaller than a month
    *.6 : KPI for the month of June
    *.7 : KPI for the month of July
    *.8 : KPI for the month of August
    FB_USER : Service scheme to avail services of Facebook and similar social networking sites
    VBC : Volume based cost - when no specific scheme is not purchased and paid as per usage

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics


In [2]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
df = pd.read_csv("train.csv")
df.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,...,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,...,0,0,NaN,1.0,NaN,710,0.0,0.0,0.0,0
2,2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,...,0,0,NaN,NaN,NaN,882,0.0,0.0,0.0,0
3,3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,...,0,0,NaN,NaN,NaN,982,0.0,0.0,0.0,0
4,4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,...,1,0,1.0,1.0,1.0,647,0.0,0.0,0.0,0


In [4]:
df.shape

(69999, 172)

In [5]:
df.describe()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
count,69999.000000,69999.0,69297.0,69297.0,69297.0,69999.000000,69999.000000,69999.000000,67231.000000,67312.000000,...,69999.000000,69999.000000,17568.000000,17865.000000,18417.000000,69999.000000,69999.000000,69999.000000,69999.00000,69999.000000
mean,34999.000000,109.0,0.0,0.0,0.0,283.134365,278.185912,278.858826,133.153275,133.894438,...,0.081444,0.085487,0.916325,0.909544,0.890319,1220.639709,68.108597,65.935830,60.07674,0.101887
std,20207.115084,0.0,0.0,0.0,0.0,334.213918,344.366927,351.924315,299.963093,311.277193,...,0.634547,0.680035,0.276907,0.286842,0.312501,952.426321,269.328659,267.899034,257.22681,0.302502
min,0.000000,109.0,0.0,0.0,0.0,-2258.709000,-1289.715000,-945.808000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.00000,0.000000
25%,17499.500000,109.0,0.0,0.0,0.0,93.581000,86.714000,84.095000,7.410000,6.675000,...,0.000000,0.000000,1.000000,1.000000,1.000000,468.000000,0.000000,0.000000,0.00000,0.000000
50%,34999.000000,109.0,0.0,0.0,0.0,197.484000,191.588000,192.234000,34.110000,32.280000,...,0.000000,0.000000,1.000000,1.000000,1.000000,868.000000,0.000000,0.000000,0.00000,0.000000
75%,52498.500000,109.0,0.0,0.0,0.0,370.791000,365.369500,369.909000,119.390000,115.837500,...,0.000000,0.000000,1.000000,1.000000,1.000000,1813.000000,0.000000,0.000000,0.00000,0.000000
max,69998.000000,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,7376.710000,8157.780000,...,33.000000,41.000000,1.000000,1.000000,1.000000,4337.000000,12916.220000,9165.600000,11166.21000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Columns: 172 entries, id to churn_probability
dtypes: float64(135), int64(28), object(9)
memory usage: 91.9+ MB


In [7]:
# Check any number of columns with NaN or missing values 
print(df.isnull().any().sum(), ' / ', len(df.columns))

125  /  172


In [8]:
# Check any number of data points with NaN
print(df.isnull().any(axis=1).sum(), ' / ', len(df))

59715  /  69999


In [9]:
df.size

12039828

In [10]:
df.drop_duplicates()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,...,0,0,NaN,NaN,NaN,1958,0.00,0.00,0.00,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,...,0,0,NaN,1.0,NaN,710,0.00,0.00,0.00,0
2,2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,...,0,0,NaN,NaN,NaN,882,0.00,0.00,0.00,0
3,3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,...,0,0,NaN,NaN,NaN,982,0.00,0.00,0.00,0
4,4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,...,1,0,1.0,1.0,1.0,647,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69994,69994,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,15.760,410.924,...,1,0,NaN,1.0,1.0,221,0.00,0.00,0.00,0
69995,69995,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,160.083,289.129,...,0,0,NaN,NaN,NaN,712,0.00,0.00,0.00,0
69996,69996,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,372.088,258.374,...,0,0,NaN,NaN,NaN,879,0.00,0.00,0.00,0
69997,69997,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,238.575,245.414,...,0,0,1.0,1.0,1.0,277,664.25,1402.96,990.97,0


In [11]:
print("Missing values in terms of Percentage")

for column in df:
    percent = 100 - (df[column].count() / len(df))*100
    if percent > 0:
        print(column + ' has ' + str(percent) + "% missing values")

Missing values in terms of Percentage
loc_og_t2o_mou has 1.0028714695924208% missing values
std_og_t2o_mou has 1.0028714695924208% missing values
loc_ic_t2o_mou has 1.0028714695924208% missing values
last_date_of_month_7 has 0.5700081429734638% missing values
last_date_of_month_8 has 1.0471578165402349% missing values
onnet_mou_6 has 3.9543422048886328% missing values
onnet_mou_7 has 3.83862626608952% missing values
onnet_mou_8 has 5.2900755725081865% missing values
offnet_mou_6 has 3.9543422048886328% missing values
offnet_mou_7 has 3.83862626608952% missing values
offnet_mou_8 has 5.2900755725081865% missing values
roam_ic_mou_6 has 3.9543422048886328% missing values
roam_ic_mou_7 has 3.83862626608952% missing values
roam_ic_mou_8 has 5.2900755725081865% missing values
roam_og_mou_6 has 3.9543422048886328% missing values
roam_og_mou_7 has 3.83862626608952% missing values
roam_og_mou_8 has 5.2900755725081865% missing values
loc_og_t2t_mou_6 has 3.9543422048886328% missing values
loc_o

In [12]:
df.dtypes.unique()

array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

In [13]:
conv_to_dt = ['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8','date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','date_of_last_rech_data_6','date_of_last_rech_data_7','date_of_last_rech_data_8']

In [14]:
for col in conv_to_dt:
    df[col] = pd.to_datetime(df[col])
    df[col] = df[col].dt.day

In [15]:
df.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [16]:
df[conv_to_dt].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   last_date_of_month_6      69999 non-null  int64  
 1   last_date_of_month_7      69600 non-null  float64
 2   last_date_of_month_8      69266 non-null  float64
 3   date_of_last_rech_6       68898 non-null  float64
 4   date_of_last_rech_7       68765 non-null  float64
 5   date_of_last_rech_8       67538 non-null  float64
 6   date_of_last_rech_data_6  17568 non-null  float64
 7   date_of_last_rech_data_7  17865 non-null  float64
 8   date_of_last_rech_data_8  18417 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 4.8 MB


In [17]:
df[conv_to_dt].head()

,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8
0,30,31.0,31.0,22.0,10.0,24.0,NaN,NaN,NaN
1,30,31.0,31.0,12.0,10.0,26.0,NaN,8.0,NaN
2,30,31.0,31.0,11.0,22.0,24.0,NaN,NaN,NaN
3,30,31.0,31.0,15.0,21.0,25.0,NaN,NaN,NaN
4,30,31.0,31.0,25.0,26.0,30.0,25.0,23.0,20.0


In [18]:
df[['last_date_of_month_7','last_date_of_month_8']].fillna(31)

,last_date_of_month_7,last_date_of_month_8
0,31.0,31.0
1,31.0,31.0
2,31.0,31.0
3,31.0,31.0
4,31.0,31.0
...,...,...
69994,31.0,31.0
69995,31.0,31.0
69996,31.0,31.0
69997,31.0,31.0


In [19]:
df['last_date_of_month_6'].fillna(30)

0        30
1        30
2        30
3        30
4        30
         ..
69994    30
69995    30
69996    30
69997    30
69998    30
Name: last_date_of_month_6, Length: 69999, dtype: int64

In [20]:
df[conv_to_dt].fillna(0)

,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8
0,30,31.0,31.0,22.0,10.0,24.0,0.0,0.0,0.0
1,30,31.0,31.0,12.0,10.0,26.0,0.0,8.0,0.0
2,30,31.0,31.0,11.0,22.0,24.0,0.0,0.0,0.0
3,30,31.0,31.0,15.0,21.0,25.0,0.0,0.0,0.0
4,30,31.0,31.0,25.0,26.0,30.0,25.0,23.0,20.0
...,...,...,...,...,...,...,...,...,...
69994,30,31.0,31.0,18.0,31.0,31.0,0.0,31.0,21.0
69995,30,31.0,31.0,28.0,31.0,27.0,0.0,0.0,0.0
69996,30,31.0,31.0,25.0,30.0,29.0,0.0,0.0,0.0
69997,30,31.0,31.0,29.0,19.0,26.0,17.0,13.0,14.0


In [21]:
df.fillna(0)

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,30,31.0,31.0,31.277,87.009,...,0,0,0.0,0.0,0.0,1958,0.00,0.00,0.00,0
1,1,109,0.0,0.0,0.0,30,31.0,31.0,0.000,122.787,...,0,0,0.0,1.0,0.0,710,0.00,0.00,0.00,0
2,2,109,0.0,0.0,0.0,30,31.0,31.0,60.806,103.176,...,0,0,0.0,0.0,0.0,882,0.00,0.00,0.00,0
3,3,109,0.0,0.0,0.0,30,31.0,31.0,156.362,205.260,...,0,0,0.0,0.0,0.0,982,0.00,0.00,0.00,0
4,4,109,0.0,0.0,0.0,30,31.0,31.0,240.708,128.191,...,1,0,1.0,1.0,1.0,647,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69994,69994,109,0.0,0.0,0.0,30,31.0,31.0,15.760,410.924,...,1,0,0.0,1.0,1.0,221,0.00,0.00,0.00,0
69995,69995,109,0.0,0.0,0.0,30,31.0,31.0,160.083,289.129,...,0,0,0.0,0.0,0.0,712,0.00,0.00,0.00,0
69996,69996,109,0.0,0.0,0.0,30,31.0,31.0,372.088,258.374,...,0,0,0.0,0.0,0.0,879,0.00,0.00,0.00,0
69997,69997,109,0.0,0.0,0.0,30,31.0,31.0,238.575,245.414,...,0,0,1.0,1.0,1.0,277,664.25,1402.96,990.97,0


In [22]:
print("Missing values in terms of Percentage")

for column in df:
    percent = 100 - (df[column].count() / len(df))*100
    if percent > 0:
        print(column + ' has ' + str(percent) + "% missing values")

Missing values in terms of Percentage
loc_og_t2o_mou has 1.0028714695924208% missing values
std_og_t2o_mou has 1.0028714695924208% missing values
loc_ic_t2o_mou has 1.0028714695924208% missing values
last_date_of_month_7 has 0.5700081429734638% missing values
last_date_of_month_8 has 1.0471578165402349% missing values
onnet_mou_6 has 3.9543422048886328% missing values
onnet_mou_7 has 3.83862626608952% missing values
onnet_mou_8 has 5.2900755725081865% missing values
offnet_mou_6 has 3.9543422048886328% missing values
offnet_mou_7 has 3.83862626608952% missing values
offnet_mou_8 has 5.2900755725081865% missing values
roam_ic_mou_6 has 3.9543422048886328% missing values
roam_ic_mou_7 has 3.83862626608952% missing values
roam_ic_mou_8 has 5.2900755725081865% missing values
roam_og_mou_6 has 3.9543422048886328% missing values
roam_og_mou_7 has 3.83862626608952% missing values
roam_og_mou_8 has 5.2900755725081865% missing values
loc_og_t2t_mou_6 has 3.9543422048886328% missing values
loc_o

loc_og_t2o_mou : local calls within same telecom circle, outgoing calls, operator t to other operator fixed lined, minutes of usage voice calls

std_og_t2o_mou : STD calls  outside the calling circle, outgoing calls, operator t to other operator fixed lined, minutes of usage voice calls

loc_ic_t2o_mou : local calls within same telecom circle, incoming calls, operator t to other operator fixed lined, minutes of usage voice calls

arpu_6 : average revenue per user, kpi for the month of june

arpu_7 : average revenue per user, kpi for the month of july

arpu_8 : average revenue per user, kpi for the month of august

onnet_mou_6 : all kind of calls within the same operator network, minutes of usage voice calls, kpi for the month 
of june

onnet_mou_7 : all kind of calls within the same operator network, minutes of usage voice calls, kpi for the month of july

onnet_mou_8 : all kind of calls within the same operator network, minutes of usage voice calls, kpi for the month of august

offnet_mou_6 : all kind of calls outside the operator T network, minutes of usage voice calls, kpi for the month of june

offnet_mou_7 : all kind of calls outside the operator T network, minutes of usage voice calls, kpi for the month of july

offnet_mou_8 : all kind of calls outside the operator T network, minutes of usage voice calls, kpi for the month of august

roam_ic_mou_6 : indicates that customer is in roaming zone during the call, incoming calls, minutes of usage voice calls, kpi for the month of june 

roam_ic_mou_7 : indicates that customer is in roaming zone during the call, incoming calls, minutes of usage voice calls, kpi for the month of july

roam_ic_mou_8 : indicates that customer is in roaming zone during the call, incoming calls, minutes of usage voice calls, kpi for the month of august

roam_og_mou_6 : indicates that customer is in roaming zone during the call, outgoing calls, minutes of usage voice alls, kpi for the month of june

roam_og_mou_7 : indicates that customer is in roaming zone during the call, outgoing calls, minutes of usage voice calls, kpi for the month of july

roam_og_mou_8 : indicates that customer is in roaming zone during the call, outgoing calls, minutes of usage voice calls, kpi for the month of august

loc_og_t2t_mou_6

loc_og_t2t_mou_7

loc_og_t2t_mou_8

loc_og_t2m_mou_6

loc_og_t2m_mou_7

loc_og_t2m_mou_8


loc_og_t2f_mou_6

loc_og_t2f_mou_7


loc_og_t2f_mou_8

loc_og_t2c_mou_6


loc_og_t2c_mou_7

loc_og_t2c_mou_8

loc_og_mou_6



loc_og_mou_7

loc_og_mou_8

std_og_t2t_mou_6



std_og_t2t_mou_7

std_og_t2t_mou_8

std_og_t2m_mou_6




std_og_t2m_mou_7

std_og_t2m_mou_8

std_og_t2f_mou_6


std_og_t2f_mou_7

std_og_t2f_mou_8

std_og_t2c_mou_6


std_og_t2c_mou_7

std_og_t2c_mou_8


std_og_mou_6


std_og_mou_7

std_og_mou_8


isd_og_mou_6

isd_og_mou_7


isd_og_mou_8

spl_og_mou_6


spl_og_mou_7

spl_og_mou_8


og_others_6

og_others_7


og_others_8

total_og_mou_6

total_og_mou_7

total_og_mou_8

loc_ic_t2t_mou_6

loc_ic_t2t_mou_7

loc_ic_t2t_mou_8

loc_ic_t2m_mou_6

loc_ic_t2m_mou_7

loc_ic_t2m_mou_8

loc_ic_t2f_mou_6

loc_ic_t2f_mou_7

loc_ic_t2f_mou_8

loc_ic_mou_6

loc_ic_mou_7

loc_ic_mou_8

std_ic_t2t_mou_6

std_ic_t2t_mou_7

std_ic_t2t_mou_8

std_ic_t2m_mou_6

std_ic_t2m_mou_7

std_ic_t2m_mou_8

std_ic_t2f_mou_6

std_ic_t2f_mou_7

std_ic_t2f_mou_8

std_ic_t2o_mou_6

std_ic_t2o_mou_7

std_ic_t2o_mou_8

std_ic_mou_6

std_ic_mou_7

std_ic_mou_8

total_ic_mou_6

total_ic_mou_7

total_ic_mou_8

spl_ic_mou_6

spl_ic_mou_7

spl_ic_mou_8

isd_ic_mou_6

isd_ic_mou_7

isd_ic_mou_8

ic_others_6

ic_others_7

ic_others_8

total_rech_num_6

total_rech_num_7

total_rech_num_8

total_rech_amt_6

total_rech_amt_7

total_rech_amt_8

max_rech_amt_6

max_rech_amt_7

max_rech_amt_8

date_of_last_rech_6

date_of_last_rech_7

date_of_last_rech_8

last_day_rch_amt_6

last_day_rch_amt_7

last_day_rch_amt_8

date_of_last_rech_data_6

date_of_last_rech_data_7

date_of_last_rech_data_8

total_rech_data_6

total_rech_data_7

total_rech_data_8

max_rech_data_6

max_rech_data_7

max_rech_data_8

count_rech_2g_6

count_rech_2g_7

count_rech_2g_8

count_rech_3g_6

count_rech_3g_7

count_rech_3g_8

av_rech_amt_data_6


av_rech_amt_data_7

av_rech_amt_data_8

vol_2g_mb_6

vol_2g_mb_7


vol_2g_mb_8

vol_3g_mb_6


vol_3g_mb_7

vol_3g_mb_8

arpu_3g_6

arpu_3g_7

arpu_3g_8

arpu_2g_6

arpu_2g_7

arpu_2g_8

night_pck_user_6

night_pck_user_7

night_pck_user_8

monthly_2g_6

monthly_2g_7

monthly_2g_8

sachet_2g_6


sachet_2g_7

sachet_2g_8

monthly_3g_6

monthly_3g_7

monthly_3g_8

sachet_3g_6

sachet_3g_7

sachet_3g_8

fb_user_6

fb_user_7

fb_user_8

aon

aug_vbc_3g

jul_vbc_3g


jun_vbc_3g

churn_probability